# Оптимизация гиперпараметров нейросети с помощью [Keras Tuner](https://github.com/keras-team/keras-tuner)

Учебный курс "[Программирование нейросетей на Python](https://www.asozykin.ru/courses/nnpython)".

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

Не забудьте подключить GPU, чтобы сеть обучалась быстрее (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU).



https://www.youtube.com/watch?v=M08yHFPKTNI&feature=youtu.be

## Гиперпараметры обучения нейронной сети

- Количество слоев нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [1]:
pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in c:\programdata\anaconda3\lib\site-packages (1.0.1)
Note: you may need to restart the kernel to use updated packages.


## Подключаем нужные пакеты

In [4]:
#%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
#from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

## Подготовка данных для обучения сети

In [5]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [6]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

## Задаем функцию создания нейронной сети

In [7]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [8]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=80,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

## Запускаем подбор гиперпараметров

Пространство поиска

In [9]:
tuner.search_space_summary()

Подбор гиперпараметров

In [10]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Epoch 1/20
188/188 [==============================] - ETA: 0s - loss: 2.6957 - accuracy: 0.09 - ETA: 2s - loss: 2.3812 - accuracy: 0.18 - ETA: 2s - loss: 2.1026 - accuracy: 0.30 - ETA: 2s - loss: 1.9120 - accuracy: 0.37 - ETA: 2s - loss: 1.7738 - accuracy: 0.43 - ETA: 2s - loss: 1.6657 - accuracy: 0.46 - ETA: 2s - loss: 1.5976 - accuracy: 0.49 - ETA: 2s - loss: 1.5300 - accuracy: 0.50 - ETA: 2s - loss: 1.4618 - accuracy: 0.52 - ETA: 2s - loss: 1.4062 - accuracy: 0.54 - ETA: 2s - loss: 1.3519 - accuracy: 0.56 - ETA: 2s - loss: 1.3071 - accuracy: 0.57 - ETA: 2s - loss: 1.2698 - accuracy: 0.58 - ETA: 1s - loss: 1.2303 - accuracy: 0.59 - ETA: 1s - loss: 1.2033 - accuracy: 0.60 - ETA: 1s - loss: 1.1744 - accuracy: 0.61 - ETA: 1s - loss: 1.1477 - accuracy: 0.61 - ETA: 1s - loss: 1.1201 - accuracy: 0.62 - ETA: 1s - loss: 1.0945 - accuracy: 0.63 - ETA: 1s - loss: 1.0750 - accuracy: 0.64 - ETA: 1s - loss: 1.0547 - accuracy: 0.64 - ETA: 1s - loss: 1.0346 - accuracy: 0.65 - ETA: 1s - loss: 1.0192

188/188 [==============================] - ETA: 0s - loss: 0.3283 - accuracy: 0.89 - ETA: 2s - loss: 0.3757 - accuracy: 0.85 - ETA: 2s - loss: 0.3805 - accuracy: 0.85 - ETA: 2s - loss: 0.3740 - accuracy: 0.85 - ETA: 2s - loss: 0.3875 - accuracy: 0.85 - ETA: 2s - loss: 0.3907 - accuracy: 0.85 - ETA: 2s - loss: 0.3874 - accuracy: 0.85 - ETA: 2s - loss: 0.3919 - accuracy: 0.85 - ETA: 2s - loss: 0.3873 - accuracy: 0.85 - ETA: 2s - loss: 0.3927 - accuracy: 0.85 - ETA: 2s - loss: 0.3983 - accuracy: 0.85 - ETA: 2s - loss: 0.4010 - accuracy: 0.85 - ETA: 2s - loss: 0.4000 - accuracy: 0.85 - ETA: 1s - loss: 0.3987 - accuracy: 0.85 - ETA: 1s - loss: 0.3983 - accuracy: 0.85 - ETA: 1s - loss: 0.3979 - accuracy: 0.85 - ETA: 1s - loss: 0.3980 - accuracy: 0.85 - ETA: 1s - loss: 0.4011 - accuracy: 0.85 - ETA: 1s - loss: 0.4001 - accuracy: 0.85 - ETA: 1s - loss: 0.3991 - accuracy: 0.85 - ETA: 1s - loss: 0.3999 - accuracy: 0.85 - ETA: 1s - loss: 0.4016 - accuracy: 0.85 - ETA: 1s - loss: 0.4018 - accuracy

188/188 [==============================] - ETA: 0s - loss: 0.4426 - accuracy: 0.85 - ETA: 2s - loss: 0.3754 - accuracy: 0.86 - ETA: 2s - loss: 0.3833 - accuracy: 0.85 - ETA: 2s - loss: 0.3691 - accuracy: 0.86 - ETA: 2s - loss: 0.3623 - accuracy: 0.86 - ETA: 2s - loss: 0.3637 - accuracy: 0.86 - ETA: 2s - loss: 0.3698 - accuracy: 0.86 - ETA: 2s - loss: 0.3661 - accuracy: 0.86 - ETA: 2s - loss: 0.3621 - accuracy: 0.86 - ETA: 2s - loss: 0.3608 - accuracy: 0.86 - ETA: 2s - loss: 0.3643 - accuracy: 0.86 - ETA: 2s - loss: 0.3593 - accuracy: 0.86 - ETA: 1s - loss: 0.3567 - accuracy: 0.86 - ETA: 1s - loss: 0.3548 - accuracy: 0.86 - ETA: 1s - loss: 0.3524 - accuracy: 0.87 - ETA: 1s - loss: 0.3499 - accuracy: 0.87 - ETA: 1s - loss: 0.3476 - accuracy: 0.87 - ETA: 1s - loss: 0.3478 - accuracy: 0.87 - ETA: 1s - loss: 0.3473 - accuracy: 0.87 - ETA: 1s - loss: 0.3479 - accuracy: 0.87 - ETA: 1s - loss: 0.3476 - accuracy: 0.87 - ETA: 1s - loss: 0.3483 - accuracy: 0.87 - ETA: 1s - loss: 0.3473 - accuracy

188/188 [==============================] - ETA: 0s - loss: 0.3405 - accuracy: 0.89 - ETA: 2s - loss: 0.3332 - accuracy: 0.88 - ETA: 2s - loss: 0.3081 - accuracy: 0.88 - ETA: 2s - loss: 0.3094 - accuracy: 0.88 - ETA: 2s - loss: 0.2984 - accuracy: 0.88 - ETA: 2s - loss: 0.3096 - accuracy: 0.88 - ETA: 2s - loss: 0.3173 - accuracy: 0.88 - ETA: 2s - loss: 0.3135 - accuracy: 0.88 - ETA: 2s - loss: 0.3109 - accuracy: 0.88 - ETA: 2s - loss: 0.3142 - accuracy: 0.88 - ETA: 2s - loss: 0.3113 - accuracy: 0.88 - ETA: 1s - loss: 0.3079 - accuracy: 0.88 - ETA: 1s - loss: 0.3054 - accuracy: 0.88 - ETA: 1s - loss: 0.3049 - accuracy: 0.88 - ETA: 1s - loss: 0.3038 - accuracy: 0.88 - ETA: 1s - loss: 0.3035 - accuracy: 0.88 - ETA: 1s - loss: 0.3029 - accuracy: 0.88 - ETA: 1s - loss: 0.3037 - accuracy: 0.88 - ETA: 1s - loss: 0.3047 - accuracy: 0.88 - ETA: 1s - loss: 0.3052 - accuracy: 0.88 - ETA: 1s - loss: 0.3059 - accuracy: 0.88 - ETA: 1s - loss: 0.3056 - accuracy: 0.88 - ETA: 1s - loss: 0.3061 - accuracy

188/188 [==============================] - ETA: 0s - loss: 0.2537 - accuracy: 0.90 - ETA: 2s - loss: 0.2898 - accuracy: 0.89 - ETA: 2s - loss: 0.2796 - accuracy: 0.89 - ETA: 2s - loss: 0.2660 - accuracy: 0.90 - ETA: 2s - loss: 0.2628 - accuracy: 0.90 - ETA: 2s - loss: 0.2765 - accuracy: 0.89 - ETA: 2s - loss: 0.2841 - accuracy: 0.89 - ETA: 2s - loss: 0.2856 - accuracy: 0.89 - ETA: 2s - loss: 0.2827 - accuracy: 0.89 - ETA: 2s - loss: 0.2821 - accuracy: 0.89 - ETA: 2s - loss: 0.2802 - accuracy: 0.89 - ETA: 2s - loss: 0.2807 - accuracy: 0.89 - ETA: 2s - loss: 0.2771 - accuracy: 0.89 - ETA: 2s - loss: 0.2787 - accuracy: 0.89 - ETA: 1s - loss: 0.2813 - accuracy: 0.89 - ETA: 1s - loss: 0.2793 - accuracy: 0.89 - ETA: 1s - loss: 0.2808 - accuracy: 0.89 - ETA: 1s - loss: 0.2809 - accuracy: 0.89 - ETA: 1s - loss: 0.2812 - accuracy: 0.89 - ETA: 1s - loss: 0.2807 - accuracy: 0.89 - ETA: 1s - loss: 0.2809 - accuracy: 0.89 - ETA: 1s - loss: 0.2817 - accuracy: 0.89 - ETA: 1s - loss: 0.2805 - accuracy

...

Epoch 1/20
188/188 [==============================] - ETA: 0s - loss: 2.5498 - accuracy: 0.08 - ETA: 1s - loss: 2.2621 - accuracy: 0.18 - ETA: 1s - loss: 2.0902 - accuracy: 0.28 - ETA: 1s - loss: 1.9793 - accuracy: 0.34 - ETA: 1s - loss: 1.8899 - accuracy: 0.38 - ETA: 1s - loss: 1.8144 - accuracy: 0.42 - ETA: 1s - loss: 1.7482 - accuracy: 0.45 - ETA: 1s - loss: 1.6899 - accuracy: 0.47 - ETA: 1s - loss: 1.6376 - accuracy: 0.49 - ETA: 1s - loss: 1.5928 - accuracy: 0.51 - ETA: 1s - loss: 1.5520 - accuracy: 0.52 - ETA: 1s - loss: 1.5199 - accuracy: 0.53 - ETA: 1s - loss: 1.4852 - accuracy: 0.54 - ETA: 0s - loss: 1.4574 - accuracy: 0.55 - ETA: 0s - loss: 1.4281 - accuracy: 0.56 - ETA: 0s - loss: 1.3996 - accuracy: 0.57 - ETA: 0s - loss: 1.3733 - accuracy: 0.58 - ETA: 0s - loss: 1.3498 - accuracy: 0.59 - ETA: 0s - loss: 1.3282 - accuracy: 0.59 - ETA: 0s - loss: 1.3058 - accuracy: 0.60 - ETA: 0s - loss: 1.2850 - accuracy: 0.60 - ETA: 0s - loss: 1.2666 - accuracy: 0.61 - ETA: 0s - loss: 1.2503

188/188 [==============================] - ETA: 0s - loss: 0.4866 - accuracy: 0.85 - ETA: 1s - loss: 0.5741 - accuracy: 0.80 - ETA: 1s - loss: 0.5455 - accuracy: 0.81 - ETA: 1s - loss: 0.5291 - accuracy: 0.82 - ETA: 1s - loss: 0.5313 - accuracy: 0.82 - ETA: 1s - loss: 0.5295 - accuracy: 0.81 - ETA: 1s - loss: 0.5317 - accuracy: 0.81 - ETA: 1s - loss: 0.5317 - accuracy: 0.81 - ETA: 1s - loss: 0.5299 - accuracy: 0.81 - ETA: 1s - loss: 0.5345 - accuracy: 0.81 - ETA: 1s - loss: 0.5301 - accuracy: 0.82 - ETA: 1s - loss: 0.5294 - accuracy: 0.82 - ETA: 1s - loss: 0.5320 - accuracy: 0.81 - ETA: 0s - loss: 0.5332 - accuracy: 0.81 - ETA: 0s - loss: 0.5333 - accuracy: 0.81 - ETA: 0s - loss: 0.5315 - accuracy: 0.81 - ETA: 0s - loss: 0.5307 - accuracy: 0.82 - ETA: 0s - loss: 0.5294 - accuracy: 0.82 - ETA: 0s - loss: 0.5286 - accuracy: 0.82 - ETA: 0s - loss: 0.5276 - accuracy: 0.82 - ETA: 0s - loss: 0.5284 - accuracy: 0.82 - ETA: 0s - loss: 0.5284 - accuracy: 0.82 - ETA: 0s - loss: 0.5303 - accuracy

188/188 [==============================] - ETA: 0s - loss: 0.4909 - accuracy: 0.81 - ETA: 1s - loss: 0.4759 - accuracy: 0.83 - ETA: 1s - loss: 0.4707 - accuracy: 0.83 - ETA: 1s - loss: 0.4679 - accuracy: 0.83 - ETA: 1s - loss: 0.4634 - accuracy: 0.83 - ETA: 1s - loss: 0.4651 - accuracy: 0.83 - ETA: 1s - loss: 0.4652 - accuracy: 0.83 - ETA: 1s - loss: 0.4660 - accuracy: 0.83 - ETA: 1s - loss: 0.4652 - accuracy: 0.83 - ETA: 1s - loss: 0.4638 - accuracy: 0.83 - ETA: 1s - loss: 0.4617 - accuracy: 0.83 - ETA: 1s - loss: 0.4631 - accuracy: 0.83 - ETA: 0s - loss: 0.4638 - accuracy: 0.83 - ETA: 0s - loss: 0.4632 - accuracy: 0.83 - ETA: 0s - loss: 0.4640 - accuracy: 0.83 - ETA: 0s - loss: 0.4651 - accuracy: 0.83 - ETA: 0s - loss: 0.4650 - accuracy: 0.83 - ETA: 0s - loss: 0.4645 - accuracy: 0.83 - ETA: 0s - loss: 0.4652 - accuracy: 0.83 - ETA: 0s - loss: 0.4647 - accuracy: 0.83 - ETA: 0s - loss: 0.4640 - accuracy: 0.83 - ETA: 0s - loss: 0.4620 - accuracy: 0.83 - ETA: 0s - loss: 0.4634 - accuracy

188/188 [==============================] - ETA: 0s - loss: 0.4467 - accuracy: 0.84 - ETA: 1s - loss: 0.4293 - accuracy: 0.84 - ETA: 1s - loss: 0.4411 - accuracy: 0.84 - ETA: 1s - loss: 0.4380 - accuracy: 0.84 - ETA: 1s - loss: 0.4376 - accuracy: 0.84 - ETA: 1s - loss: 0.4324 - accuracy: 0.84 - ETA: 1s - loss: 0.4273 - accuracy: 0.84 - ETA: 1s - loss: 0.4313 - accuracy: 0.84 - ETA: 1s - loss: 0.4316 - accuracy: 0.84 - ETA: 1s - loss: 0.4302 - accuracy: 0.84 - ETA: 1s - loss: 0.4301 - accuracy: 0.84 - ETA: 1s - loss: 0.4296 - accuracy: 0.84 - ETA: 0s - loss: 0.4296 - accuracy: 0.84 - ETA: 0s - loss: 0.4299 - accuracy: 0.84 - ETA: 0s - loss: 0.4337 - accuracy: 0.84 - ETA: 0s - loss: 0.4342 - accuracy: 0.84 - ETA: 0s - loss: 0.4351 - accuracy: 0.84 - ETA: 0s - loss: 0.4342 - accuracy: 0.84 - ETA: 0s - loss: 0.4340 - accuracy: 0.84 - ETA: 0s - loss: 0.4322 - accuracy: 0.84 - ETA: 0s - loss: 0.4320 - accuracy: 0.84 - ETA: 0s - loss: 0.4327 - accuracy: 0.84 - ETA: 0s - loss: 0.4355 - accuracy

INFO:tensorflow:Oracle triggered exit


## Выбираем лучшую модель

In [ ]:
tuner.results_summary()

Получаем три лучших модели

In [ ]:
models = tuner.get_best_models(num_models=3)

Оцениваем качество модели на тестовых данных

In [ ]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 896)               703360    
_________________________________________________________________
dense_1 (Dense)              (None, 448)               401856    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                4490      
Total params: 1,109,706
Trainable params: 1,109,706
Non-trainable params: 0
_________________________________________________________________
10000/10000 [==============================] - 1s 92us/sample - loss: 0.3721 - accuracy: 0.8941

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 608)               477280    
_____________________________________________________________